# Bloodhound Playbook - Most Privileged assets and Paths to DA
-----------------------------------------
* **Reference:** https://posts.specterops.io/visualizing-bloodhound-data-with-powerbi-part-1-ba8ea4908422

## Import Libraries

In [2]:
from py2neo import Graph
import plotly.graph_objects as go
import altair as alt
alt.renderers.enable('notebook')

RendererRegistry.enable('notebook')

## Initialize BloodHound neo4j Database Connection

In [3]:
g = Graph("bolt://127.0.0.1:7687", auth=("neo4j", "neo4j"))
g

<Graph database=<Database uri='bolt://127.0.0.1:7687' secure=False user_agent='py2neo/4.2.0 neobolt/1.7.15 Python/3.7.4-final-0 (win32)'> name='data'>

## Retrieve Security Groups with Local Admin Rights over Computers

Now we’ll construct the Cypher query that tells us the name of each security group in Active Directory and the number of computers that group has local admin rights on.

In [39]:
sg_computers_df = g.run("""
MATCH (g:Group)
OPTIONAL MATCH (g)-[:AdminTo]->(c1:Computer)
OPTIONAL MATCH (g)-[:MemberOf*1..]->(:Group)-[:AdminTo]->(c2:Computer)
WITH g, COLLECT(c1) + COLLECT(c2) AS tempVar
UNWIND tempVar AS computers
RETURN g.name AS GroupName,COUNT(DISTINCT(computers)) AS AdminRightCount
ORDER BY AdminRightCount DESC
""").to_data_frame()

In [ ]:
sg_computers_df[:10]

## Create Horizontal Bar Chart: Most Privileged Active Directory Security Groups

In [ ]:
bars = alt.Chart(sg_computers_df.head(30), title="Most Privileged Active Directory Security Groups").mark_bar().encode(
    x='AdminRightCount:Q',
    y=alt.Y(
        "GroupName:N",
        sort=alt.EncodingSortField(
            field="AdminRightCount",
            order="descending"
        )
    )
)

text = bars.mark_text(
    align='left',
    baseline='middle',
    dx=3
).encode(
    text='AdminRightCount:Q'
)

(bars + text).properties(height=300)

## Create Percentage Chart: Domain Users with a path to Domain Admin

In [19]:
users_to_da = g.run("""
MATCH (totalUsers:User {domain:'TESTLAB.NET'})
MATCH p=shortestPath((UsersWithPath:User {domain:'TESTLAB.NET'})-[r*1..]->(g:Group {name:'DOMAIN ADMINS@TESTLAB.NET'}))
WITH COUNT(DISTINCT(totalUsers)) as totalUsers, COUNT(DISTINCT(UsersWithPath)) as UsersWithPath
RETURN 100.0 * UsersWithPath / totalUsers AS percentUsersToDA
""").to_data_frame()

In [ ]:
users_to_da

In [ ]:
value_df = users_to_da['percentUsersToDA'].values[0]
value_df

In [ ]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = (value_df),
    mode = "gauge+number",
    title = {'text': "Percentage of Users with a Path to Domain Admin"},
    gauge = {'axis': {'range': [None, 100]},
             'steps' : [{'range': [0, 250], 'color': "lightgray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 490}}))

fig.show()
         

## Removing two groups with highest no of paths (blacklist in bloodhound)

In [24]:
users_to_da_wo_grps = g.run("""
MATCH (totalUsers:User {domain:'TESTLAB.NET'})
MATCH p=shortestPath((UsersWithPath:User {domain:'TESTLAB.NET'})-[r*1..]->(g:Group {name:'DOMAIN ADMINS@TESTLAB.NET'}))
WITH COUNT(DISTINCT(totalUsers)) as totalUsers, COUNT(DISTINCT(UsersWithPath)) as UsersWithPath
RETURN 100.0 * UsersWithPath / totalUsers AS percentUsersToDA
""").to_data_frame()

In [ ]:
users_to_da_wo_grps

In [ ]:
value_df = users_to_da_wo_grps['percentUsersToDA'].values[0]
value_df

In [ ]:
fig = go.Figure(go.Indicator(
    domain = {'x': [0, 1], 'y': [0, 1]},
    value = (value_df),
    mode = "gauge+number",
    title = {'text': "Percentage of Users with a Path to Domain Admin"},
    gauge = {'axis': {'range': [None, 100]},
             'steps' : [{'range': [0, 250], 'color': "lightgray"}],
             'threshold' : {'line': {'color': "red", 'width': 4}, 'thickness': 0.75, 'value': 490}}))

fig.show()